In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/wine-dataset-for-clustering/wine-clustering.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()
data=df.copy()
data.head()

## Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
data[data.columns]=sc.fit_transform(data)

In [ ]:
data

In [ ]:
data[data.columns]

## Applying PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(data)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid", rc={'figure.figsize':(9,6)}, font_scale=2)
plt.scatter(x=principalComponents[:,0], y=principalComponents[:,1], color="black", lw=2)
plt.xlabel("Principle Component 1")
plt.ylabel("Principle Component 2")
plt.title("Strongest Principle Components")
plt.show()

## Finding the best number for K

In [ ]:
#Visualizing the ELBOW method to get the optimal value of K 
from sklearn.cluster import KMeans
wcss=[] # within cluster sum of sqaures
for i in range(1,11):
    kmeans = KMeans(n_clusters= i, init='k-means++', random_state=0)
    kmeans.fit(data)
    wcss.append(kmeans.inertia_)

#inertia_ is the formula used to segregate the data points into clusters
import matplotlib.pyplot as plt
plt.plot(range(1,11), wcss)
plt.title('The Elbow Method')
plt.xlabel('no of clusters')
plt.ylabel('wcss')
plt.show()

## Applying K-Means

In [ ]:
kmeans= KMeans(n_clusters=3, random_state=17, init='k-means++')
kmeans_labels= kmeans.fit_predict(data)
centroids = kmeans.cluster_centers_

In [ ]:
centroids_pca = pca.transform(centroids)
pd.Series(kmeans_labels).value_counts()

In [ ]:
centroids_df = pd.DataFrame(sc.inverse_transform(centroids), columns= df.columns)
centroids_df

In [ ]:
sns.set(style='darkgrid', rc={'figure.figsize':(12,8)},font_scale=2)
plt.scatter(x=principalComponents[:,0], y=principalComponents[:,1], c=kmeans_labels, cmap="CMRmap", lw=4)
plt.scatter(x=centroids_pca[:,0], y=centroids_pca[:,1], marker="+", s=500, linewidths=3, lw=4, color="blue", zorder=10)
plt.xlabel("Principle Component 1")
plt.ylabel("Principle Component 2")
plt.title("Clustered Data")

In [ ]:
from sklearn.metrics import silhouette_score
silhouette = silhouette_score(data, kmeans.labels_, metric='euclidean')

In [ ]:
silhouette

In [ ]:
sns.set(style="darkgrid", font_scale=1.5, rc={'figure.figsize':(20,20)})
ax=df.hist(bins=20, color="green")